In [1]:
import numpy as np

In [2]:
class BinomialTree:
    
    def __init__(self, S0, T, vol, N, r, c, u=None):
        self.S0 = S0 # initial stock price
        self.T = T # time to maturity
        self.N = N # number of periods
        self.dt = T / N # time increment
        self.vol = vol # volatility
        self.r = r # risk-free rate
        self.c = c # dividend yield
        if u:
            self.u = u # up factor
        else:
            self.u = np.exp(self.vol * np.sqrt(self.T / self.N))
        self.d = 1 / self.u # down factor
        self.q = (np.exp((self.r - self.c) * self.T / self.N) - self.d) / (self.u - self.d) # risk-neutral probability
    
    def stock_price(self, t, i):
        assert 0<= t <= self.N, 't must be between 0 and N'
        assert 0<= i <= t, 'i must be between 0 and t'
        return self.S0 * self.u ** (t - i) * self.d ** i
    
    def stock_tree(self):
        return [[self.stock_price(t, i) for i in range(t + 1)] for t in range(self.N + 1)]
    
    def stock_tree_print(self):
        print('Stock Price Tree')
        print('----------------')
        for t in range(self.N + 1):
            print('t =', t, ':')
            for i in range(t + 1):
                print("{:.2f}".format(self.stock_price(t, i)), end=' ')
            print('\n')
    
    def option_price(self, t, i, K, m, type_, class_):
        assert type_ in ['call', 'put'], 'type_ must be either call or put'
        assert class_ in ['european', 'american'], 'class_ must be either european or american'
        assert 0<= m <= self.N, 'm must be between 0 and N'
        assert 0<= t <= m, 't must be between 0 and m'
        assert 0<= i <= t, 'i must be between 0 and t'
        
        if t == m:
            if type_ == 'call':
                return max(self.stock_price(t, i) - K, 0)
            elif type_ == 'put':
                return max(K - self.stock_price(t, i), 0)
        else:
            up = self.option_price(t + 1, i, K, m, type_, class_)
            down = self.option_price(t + 1, i + 1, K, m, type_, class_)
            discount_price = np.exp(-self.r * self.T / self.N) * (self.q * up + (1 - self.q) * down)
            if class_ == 'american':
                if type_ == 'call':
                    return max(self.stock_price(t, i) - K, discount_price)
                elif type_ == 'put':
                    return max(K - self.stock_price(t, i), discount_price)
            elif class_ == 'european':
                return discount_price
    
    def option_tree(self, K, m, type_, class_):
        return [[self.option_price(t, i, K, m, type_, class_) for i in range(t + 1)] for t in range(self.N + 1)]
    
    def option_tree_print(self, K, m, type_, class_):
        print('Option Price Tree')
        print('-----------------')
        for t in range(m + 1):
            print('t =', t, ':')
            for i in range(t + 1):
                print("{:.2f}".format(self.option_price(t, i, K, m, type_, class_)), end=' ')
            print('\n')
            
    def early_exercise(self, K, m, type_, class_):
        assert class_ == 'american', 'early exercise only applies to american options'
        print('Early Exercise')
        print('--------------')
        for t in range(m):
            print('t =', t, ':')
            for i in range(t + 1):
                up = self.option_price(t + 1, i, K, m, type_, class_)
                down = self.option_price(t + 1, i + 1, K, m, type_, class_)
                discount_price = np.exp(-self.r * self.T / self.N) * (self.q * up + (1 - self.q) * down)
                if type_ == 'call':
                    print(int(self.stock_price(t, i) - K > discount_price), end=' ')
                elif type_ == 'put':
                    print(int(K - self.stock_price(t, i) > discount_price), end=' ')
            print('\n')

In [20]:
# Lattice Parameters
S0 = 100 # initial stock price
T = 0.25 # maturity
vol = 0.3 # volatility
N = 15 # number of time steps
r = 0.02 # risk-free rate
c = 0.01 # dividend yield

# Option Parameters
K = 110 # strike price
m = 10 # maturity (in number of periods)
type_ = 'call' # option type
class_ = 'american' # option class

In [21]:
test = BinomialTree(S0, T, vol, N, r, c)

In [28]:
test.u

1.0394896104013376

In [24]:
test.stock_tree_print()

Stock Price Tree
----------------
t = 0 :
100.00 

t = 1 :
103.95 96.20 

t = 2 :
108.05 100.00 92.55 

t = 3 :
112.32 103.95 96.20 89.03 

t = 4 :
116.76 108.05 100.00 92.55 85.65 

t = 5 :
121.37 112.32 103.95 96.20 89.03 82.39 

t = 6 :
126.16 116.76 108.05 100.00 92.55 85.65 79.26 

t = 7 :
131.14 121.37 112.32 103.95 96.20 89.03 82.39 76.25 

t = 8 :
136.32 126.16 116.76 108.05 100.00 92.55 85.65 79.26 73.36 

t = 9 :
141.70 131.14 121.37 112.32 103.95 96.20 89.03 82.39 76.25 70.57 

t = 10 :
147.30 136.32 126.16 116.76 108.05 100.00 92.55 85.65 79.26 73.36 67.89 

t = 11 :
153.12 141.70 131.14 121.37 112.32 103.95 96.20 89.03 82.39 76.25 70.57 65.31 

t = 12 :
159.16 147.30 136.32 126.16 116.76 108.05 100.00 92.55 85.65 79.26 73.36 67.89 62.83 

t = 13 :
165.45 153.12 141.70 131.14 121.37 112.32 103.95 96.20 89.03 82.39 76.25 70.57 65.31 60.44 

t = 14 :
171.98 159.16 147.30 136.32 126.16 116.76 108.05 100.00 92.55 85.65 79.26 73.36 67.89 62.83 58.15 

t = 15 :
178.77 165.45 153.

In [25]:
test.option_tree_print(K, m, type_, class_)

Option Price Tree
-----------------
t = 0 :
1.65 

t = 1 :
2.58 0.74 

t = 2 :
3.96 1.24 0.25 

t = 3 :
5.93 2.06 0.45 0.05 

t = 4 :
8.62 3.33 0.82 0.10 0.00 

t = 5 :
12.10 5.24 1.47 0.20 0.00 0.00 

t = 6 :
16.35 7.99 2.59 0.40 0.00 0.00 0.00 

t = 7 :
21.19 11.67 4.42 0.81 0.00 0.00 0.00 0.00 

t = 8 :
26.35 16.19 7.29 1.64 0.00 0.00 0.00 0.00 0.00 

t = 9 :
31.72 21.16 11.38 3.33 0.00 0.00 0.00 0.00 0.00 0.00 

t = 10 :
37.30 26.32 16.16 6.76 0.00 0.00 0.00 0.00 0.00 0.00 0.00 

